# Notebook 01 — ETL (Extração, Transformação e Carga)
Este notebook carrega o dataset `tips` do seaborn, salva o raw CSV, realiza limpeza/transformação (One-Hot Encoding) e salva os dados processados (CSV + .npy).

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

# paths
RAW_DIR = "data/raw"
PROCESSED_DIR = "data/processed"
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

In [ ]:
# carregar dataset original do seaborn
df = sns.load_dataset("tips")

# visualizar
display(df.head())
print(df.shape)

# salvar raw CSV
raw_path = os.path.join(RAW_DIR, "tips_raw.csv")
df.to_csv(raw_path, index=False)
print(f"Saved raw csv to: {raw_path}")

In [ ]:
# info e missing
print("Info:")
print(df.info())
print("\nMissing values per column:")
print(df.isnull().sum())

# estatísticas
display(df.describe(include='all'))

In [ ]:
# No caso do dataset 'tips' padrão não há nulos, mas segue um bloco para eventuais tratamentos:
# Remover duplicatas (se existirem)
initial_len = len(df)
df = df.drop_duplicates().reset_index(drop=True)
print(f"Dropped {initial_len - len(df)} duplicate rows (if any).")

# Converter tipos se necessário (exemplo)
# df['size'] = df['size'].astype(int)

In [ ]:
# Escolher colunas categóricas
cat_cols = ['sex', 'smoker', 'day', 'time']

# One-hot encoding (mantendo todas as colunas)
df_encoded = pd.get_dummies(df, columns=cat_cols, prefix=cat_cols)

# Reordenar colunas - colocar total_bill, tip, size primeiro
cols = df_encoded.columns.tolist()
ordered_cols = ['total_bill', 'tip', 'size'] + [c for c in cols if c not in ('total_bill', 'tip', 'size')]
df_encoded = df_encoded[ordered_cols]

display(df_encoded.head())

In [ ]:
processed_csv = os.path.join(PROCESSED_DIR, "tips_clean.csv")
df_encoded.to_csv(processed_csv, index=False)
print(f"Saved processed csv to: {processed_csv}")

# salvar features (X) e target (y) como .npy (opcional)
X = df_encoded.drop(columns=['tip'])
y = df_encoded['tip']

np.save(os.path.join(PROCESSED_DIR, "features.npy"), X.values)
np.save(os.path.join(PROCESSED_DIR, "target.npy"), y.values)

print("Saved features.npy and target.npy")